In [354]:
"""
Calculates the amount of anomalous data points in the NZERTF data
"""
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest

import DataRetriever as dr

retriever = dr.DataRetriever()

year_two = retriever.get_data("All-Subsystems-minute-Year2.pkl")


,TimeStamp_Count,DHW_WaterTempDishwasher,DHW_WaterTempWasherCold,DHW_WaterTempWasherHot,DHW_WaterTempMBASinkRCold,DHW_WaterTempKSinkMix,DHW_WaterTempSHWTank80galOut,DHW_WaterTempSHWHX80galOut,DHW_SHWGlycolTempIn,DHW_WaterTempMBASinkRHot
count,519604.000000,5.196040e+05,5.196040e+05,5.196040e+05,5.196040e+05,5.196040e+05,5.196040e+05,5.196040e+05,5.196040e+05,5.196040e+05
mean,259802.500000,-6.219481e+88,-2.050354e+86,-8.140542e+84,-9.978438e+89,-9.806032e+84,-8.226175e+84,-2.717354e+87,-1.605094e+85,-1.001487e+90
std,149996.898974,6.999865e+89,2.955900e+88,5.867993e+87,2.167977e+90,7.068537e+87,5.929720e+87,1.493309e+89,1.157009e+88,2.175824e+90
min,1.000000,-9.036153e+90,-4.295621e+90,-4.229858e+90,-1.187051e+91,-5.095254e+90,-4.274354e+90,-8.272135e+90,-8.340133e+90,-1.193160e+91
25%,129901.750000,2.061378e+01,2.110233e+01,2.124000e+01,2.065757e+01,2.122916e+01,2.263928e+01,2.104551e+01,1.923127e+01,2.102846e+01
50%,259802.500000,2.199918e+01,2.158445e+01,2.169624e+01,2.283321e+01,2.280231e+01,3.057006e+01,2.756492e+01,2.197240e+01,2.378513e+01
75%,389703.250000,2.281471e+01,2.204376e+01,2.229288e+01,2.392276e+01,3.201328e+01,3.859792e+01,3.802486e+01,3.141633e+01,2.466425e+01
max,519604.000000,8.510165e+02,2.627878e+01,5.360049e+01,2.810738e+02,4.914361e+01,7.189244e+01,7.462323e+01,9.173174e+01,3.760387e+02


In [349]:
year_two.drop(['Timestamp', 'TimeStamp_Count', 'DayOfWeek', 'DHW_WaterTempDishwasher', 'DHW_WaterTempWasherCold',
               'DHW_WaterTempWasherHot', 'DHW_WaterTempMBASinkRCold', 'DHW_WaterTempKSinkMix','DHW_WaterTempSHWTank80galOut', 'DHW_WaterTempSHWHX80galOut', 'DHW_SHWGlycolTempIn', 'DHW_WaterTempMBASinkRHot'], axis=1, inplace=True)
year_two.fillna(0, inplace=True)

In [350]:
iso_forest = IsolationForest(n_estimators=100, max_samples='auto', contamination='auto', max_features=1.0,
                             bootstrap=False, n_jobs=1, random_state=1, warm_start=False)

iso_forest.fit(year_two.values)
predictions = iso_forest.predict(year_two.values)
amount_anomalous = len(np.where(predictions < 0)[0])
amount_not_anomalous = len(np.where(predictions > 0)[0])

proportion_anomalous = amount_anomalous / len(year_two)
proportion_not_anomalous = amount_not_anomalous / len(year_two)
print(f'Proportion anomalous: {amount_anomalous}/{len(year_two)} = {round(proportion_anomalous * 100, 2)}%')
print(f'Proportion not anomalous: {amount_not_anomalous}/{len(year_two)} = {round(proportion_not_anomalous * 100, 2)}%')

anomalous_indeces = np.where(predictions == -1)[0].tolist()
print(year_two.iloc[anomalous_indeces])

Proportion anomalous: 28678/519604 = 5.52%
Proportion not anomalous: 490926/519604 = 94.48%
        Load_LatentHeatWaterVolume  Load_RefrigeratorTemp  \
543                       0.585863               5.481648   
553                       0.596432               5.115619   
554                       0.596432               5.010662   
559                       0.603036               4.646350   
560                       0.603036               4.658486   
...                            ...                    ...   
519541                    1.685596               5.393046   
519542                    1.685596               5.443258   
519543                    1.688899               5.349060   
519548                    1.695503               5.085212   
519549                    1.695503               5.069857   

        Load_StatusBA1Lights  Load_StatusKitchenLightsA  \
543                      0.0                        1.0   
553                      0.0                        1.0  